This code is for Example 2b when $A = -100(\cos( f_1)\sin(f_2), \sin(f_1)\cos(f_2)))$, where $f_1 = 5\pi \sin(x^2 + y^2), f_2 =5\pi \cos(x^2 + y^2)$

Add scalar field V

# This section is to calculate the eigenpairs for H(A, V)

In [13]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
import random

start1 = time.time()
# Generate 16x16 congruent squares
n =16
list_S = []
for j in range(n):
    for i in range(n):
        S = MoveTo(-1+2*i/n, -1+2*j/n).Rectangle(2/n, 2/n).Face()
        S.faces.name="S"+ str(i+1+j*n)
        S.edges.name="inner"
        if j==0: S.edges.Min(Y).name = "bottom"
        if j==n-1: S.edges.Max(Y).name = "top"
        if i==0: S.edges.Min(X).name = "left"
        if i==n-1: S.edges.Max(X).name = "right"
        list_S.append(S)
geo = Glue(list_S)
mesh = Mesh(OCCGeometry(geo,dim=2).GenerateMesh(maxh=0.05)) 
# Assign random numbers in [0, V_max] to each square 
tk_coef = {}
random.seed(1)
# Fix scale to 100, 500, and 1000 for V_max
scale =100 
for i in range(n*n):
    tkC = random.uniform(0, 1)*scale
    tk_coef['S'+str(i+1)] = tkC
tk =mesh.MaterialCF(tk_coef)
V = tk
# Draw(V, mesh)

a = 100
A = CF((-a*cos(5*pi*sin(x**2 + y**2))*sin(5*pi*cos(x**2 + y**2)), -a*sin(5*pi*sin(x**2 + y**2))*cos(5*pi*cos(x**2 + y**2))))
X = H1(mesh, order=3, complex=True, dirichlet = 'bottom|top|left|right')
# X.ndof
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx + V* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

# Use FEAST algorithm for computing eigenvalues
seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# scale=100: When h=0.01, radius = 55, center = 192; When h=0.03, radius = 52, center = 207; When h=0.05, radius = 56, center = 224
# scale=500: When h=0.01, radius = 82, center = 306; When h=0.03, radius = 80, center = 322; When h=0.05, radius = 87, center = 347
# scale=1000: When h=0.01, radius = 97, center = 393; When h=0.03, radius = 99, center = 411; When h=0.05, radius = 103, center = 432
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius=56,
                   center= 224,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=56, Center=224+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +277.259+17.305j
SpectralProjNG:   Factorizing at z = +256.916+45.305j
SpectralProjNG:   Factorizing at z = +224.000+56.000j
SpectralProjNG:   Factorizing at z = +191.084+45.305j
SpectralProjNG:   Factorizing at z = +170.741+17.305j
SpectralProjNG:   Factorizing at z = +170.741-17.305j
SpectralProjNG:   Factorizing at z = +191.084-45.305j
SpectralProjNG:   Factorizing at z = +224.000-56.000j
SpectralProjNG:   Factorizing at z = +256.916-45.305j
SpectralProjNG:   Factorizing at z = +277.259-17.305j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [168.80846389 198.29499759 241.37377335 265.96531587 272.17285864
 278.1273345  295.93342473 320.0333116 ]
   Relative Hausdorff distance from pri

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the degree of freedom 

In [14]:
X.ndof 

21025

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [15]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

(49.00261421993348+0j) (49.381868703015364+0j)
(65.03109216532069+0j) (65.96733509913886+0j)
(64.61763868102179+0j) (65.62655923563356+0j)
(67.31566831284351+0j) (68.20337704564683+0j)
(50.45945322554646+0j) (50.899105380947525+0j)
(50.44743508365226+0j) (50.87793601057124+0j)


# This section is to calculate the eigenpairs for H(F, V)

In [18]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
import random
start1 = time.time()
n =16
list_S = []
for j in range(n):
    for i in range(n):
        S = MoveTo(-1+2*i/n, -1+2*j/n).Rectangle(2/n, 2/n).Face()
        S.faces.name="S"+ str(i+1+j*n)
        S.edges.name="inner"
        if j==0: S.edges.Min(Y).name = "bottom"
        if j==n-1: S.edges.Max(Y).name = "top"
        if i==0: S.edges.Min(X).name = "left"
        if i==n-1: S.edges.Max(X).name = "right"
        list_S.append(S)

geo = Glue(list_S)
mesh = Mesh(OCCGeometry(geo,dim=2).GenerateMesh(maxh=0.05)) 
# Draw(mesh)
tk_coef = {}
random.seed(1)
scale = 100
for i in range(n*n):
    tkC = random.uniform(0, 1)*scale
    tk_coef['S'+str(i+1)] = tkC
tk =mesh.MaterialCF(tk_coef)
V = tk
# Draw(V, mesh)

a = 100
A = CF((-a*cos(5*pi*sin(x**2 + y**2))*sin(5*pi*cos(x**2 + y**2)), -a*sin(5*pi*sin(x**2 + y**2))*cos(5*pi*cos(x**2 + y**2))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N
# fesm.ndof
u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)
Xeig =  H1(mesh, order=3, complex=True, dirichlet = 'bottom|top|left|right')
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx + V* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

seed =1
npts=10
nspan=8
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# Scale=100:  radius = 55, center = 192
# Scale=500: radius = 82, center = 306
# Scale=1000: radius = 98, center = 393
P = SpectralProjNG(Xeig,
                   a2.mat,
                   b2.mat,
                   radius=55,
                   center=192,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
# print('Prepare time', start2-start1)
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y1 = Y.gridfun()
Draw(y1)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=55, Center=192+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +244.308+16.996j
SpectralProjNG:   Factorizing at z = +224.328+44.496j
SpectralProjNG:   Factorizing at z = +192.000+55.000j
SpectralProjNG:   Factorizing at z = +159.672+44.496j
SpectralProjNG:   Factorizing at z = +139.692+16.996j
SpectralProjNG:   Factorizing at z = +139.692-16.996j
SpectralProjNG:   Factorizing at z = +159.672-44.496j
SpectralProjNG:   Factorizing at z = +192.000-55.000j
SpectralProjNG:   Factorizing at z = +224.328-44.496j
SpectralProjNG:   Factorizing at z = +244.308-16.996j

=========== Starting FEAST iterations ===========
Trying with 8 vectors:

 ITERATION 1 with 8 vectors
   Real part of computed eigenvalues:
   [137.76134285 148.05250037 198.79385037 216.58879705 241.75692953
 246.02447235 250.74498632 251.34361992]
   Relative Hausdorff distance from pri

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \phi$ and $F\phi$

In [19]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

(30.693800405811395+0j) (30.05190806573415+0j)
(34.64377396957469+0j) (34.00796654101724+0j)
(34.66106802493703+0j) (33.560605065684136+0j)
(41.26443027562706+0j) (40.15850593874018+0j)
(38.21077391992729+0j) (36.701191492640575+0j)
(21.483177704267224+0j) (20.331025929539162+0j)
